In [11]:
import random
import numpy as np

def simulate_mms_system(S, n, lambda_val, mu_val, num_iterations):
    """
    Моделирует систему массового обслуживания с S каналами и очередью длиной n.

    Args:
        S (int): Количество каналов обслуживания.
        n (int): Длина очереди.
        lambda_val (float): Интенсивность поступления заявок.
        mu_val (float): Интенсивность обслуживания заявок.
        num_iterations (int): Количество итераций моделирования.

    Returns:
        tuple: (среднее количество обслуженных клиентов,
                среднее количество отклонённых клиентов,
                вероятность отказа)
    """

    total_served = 0
    total_rejected = 0

    for _ in range(num_iterations):
        # Состояние системы: количество занятых каналов, количество заявок в очереди
        current_state = (0, 0)
        served_in_iteration = 0
        rejected_in_iteration = 0

        # Время до следующего события (поступление или окончание обслуживания)
        time_to_next_arrival = random.expovariate(lambda_val)
        time_to_next_service_completion = float('inf') if current_state[0] == 0 else random.expovariate(mu_val)

        # Моделирование событий во времени
        current_time = 0
        while current_time < 1000:  # Моделируем в течение некоторого времени
            if time_to_next_arrival <= time_to_next_service_completion:
                # Событие: поступление заявки
                current_time += time_to_next_arrival
                time_to_next_service_completion -= time_to_next_arrival

                if current_state[0] < S:
                    # Заявка поступает в свободный канал
                    current_state = (current_state[0] + 1, current_state[1])
                    served_in_iteration += 1
                    time_to_next_service_completion = random.expovariate(mu_val)
                elif current_state[1] < n:
                    # Заявка поступает в очередь
                    current_state = (current_state[0], current_state[1] + 1)
                else:
                    # Заявка отклонена (очередь полна)
                    rejected_in_iteration += 1

                time_to_next_arrival = random.expovariate(lambda_val)

            else:
                # Событие: окончание обслуживания
                current_time += time_to_next_service_completion
                time_to_next_arrival -= time_to_next_service_completion

                if current_state[1] > 0:
                    # Заявка из очереди переходит в освободившийся канал
                    current_state = (current_state[0], current_state[1] - 1)
                    served_in_iteration += 1
                    time_to_next_service_completion = random.expovariate(mu_val)
                else:
                    # Канал освобождается
                    current_state = (current_state[0] - 1, current_state[1])
                    if current_state[0] > 0:
                        time_to_next_service_completion = random.expovariate(mu_val)
                    else:
                        time_to_next_service_completion = float('inf')

        total_served += served_in_iteration
        total_rejected += rejected_in_iteration

    average_served = total_served / num_iterations
    average_rejected = total_rejected / num_iterations
    total_arrivals = total_served + total_rejected  # Приближенное количество поступивших заявок
    rejection_probability = average_rejected / total_arrivals if total_arrivals > 0 else 0

    return average_served, average_rejected, rejection_probability

# Входные параметры
S_values = [2, 4]
n_values = [10, 20]
lambda_values = [0.7, 0.3]
mu_values = [1.2, 0.5]
num_iterations = 100

# Генерация комбинаций параметров
combinations = []
for s in S_values:
    for n in n_values:
        for lambda_val in lambda_values:
            for mu_val in mu_values:
                combinations.append({'S': s, 'n': n, 'lambda': lambda_val, 'mu': mu_val})

# Выбираем 6 уникальных комбинаций (можно выбрать любые 6)
selected_combinations = [
    {'S': 2, 'n': 10, 'lambda': 0.7, 'mu': 1.2},
    {'S': 4, 'n': 10, 'lambda': 0.7, 'mu': 0.5},
    {'S': 2, 'n': 20, 'lambda': 0.3, 'mu': 1.2},
    {'S': 4, 'n': 20, 'lambda': 0.3, 'mu': 0.5},
    {'S': 2, 'n': 10, 'lambda': 0.3, 'mu': 0.5},
    {'S': 4, 'n': 20, 'lambda': 0.7, 'mu': 1.2}
]


print("Результаты моделирования системы массового обслуживания:")
print("-" * 60)

for params in selected_combinations:
    S = params['S']
    n = params['n']
    lambda_val = params['lambda']
    mu_val = params['mu']

    avg_served, avg_rejected, prob_rejection = simulate_mms_system(S, n, lambda_val, mu_val, num_iterations)

    print(f"Набор параметров: S={S}, n={n}, λ={lambda_val}, μ={mu_val}")
    print(f"  Среднее количество обслуженных клиентов: {avg_served:.2f}")
    print(f"  Среднее количество отклонённых клиентов: {avg_rejected:.2f}")
    print(f"  Вероятность отказа: {prob_rejection:.4f}")
    print("-" * 60)

Результаты моделирования системы массового обслуживания:
------------------------------------------------------------
Набор параметров: S=2, n=10, λ=0.7, μ=1.2
  Среднее количество обслуженных клиентов: 703.00
  Среднее количество отклонённых клиентов: 0.61
  Вероятность отказа: 0.0000
------------------------------------------------------------
Набор параметров: S=4, n=10, λ=0.7, μ=0.5
  Среднее количество обслуженных клиентов: 499.76
  Среднее количество отклонённых клиентов: 195.06
  Вероятность отказа: 0.0028
------------------------------------------------------------
Набор параметров: S=2, n=20, λ=0.3, μ=1.2
  Среднее количество обслуженных клиентов: 301.81
  Среднее количество отклонённых клиентов: 0.00
  Вероятность отказа: 0.0000
------------------------------------------------------------
Набор параметров: S=4, n=20, λ=0.3, μ=0.5
  Среднее количество обслуженных клиентов: 302.32
  Среднее количество отклонённых клиентов: 0.00
  Вероятность отказа: 0.0000
---------------------

In [15]:
import math

def calculate_queue_metrics(S, lambda_, mu, n):
    """
    Расчет характеристик СМО M/M/S/n.
    S - число каналов, lambda_ - интенсивность входного потока,
    mu - интенсивность обслуживания, n - длина очереди.
    """
    rho = lambda_ / (S * mu)  # Интенсивность нагрузки на канал
    p0 = 0  # Вероятность простоя системы (все каналы свободны)
    
    # Вычисление p0 (вероятность простоя)
    for k in range(S):
        p0 += (math.pow(S * rho, k) / math.factorial(k))
    for k in range(S, S + n + 1):
        p0 += (math.pow(S * rho, k) / (math.factorial(S) * math.pow(S, k - S)))
    p0 = 1 / p0 if p0 != 0 else 0
    
    # Вероятность отказа (заявка уходит, если очередь полна)
    p_refuse = (math.pow(S * rho, S + n) / (math.factorial(S) * math.pow(S, n))) * p0
    
    # Среднее число заявок в очереди
    L_queue = 0
    for k in range(1, n + 1):
        L_queue += k * (math.pow(S * rho, S + k) / (math.factorial(S) * math.pow(S, k))) * p0
    
    # Среднее число занятых каналов
    L_busy = lambda_ * (1 - p_refuse) / mu
    
    # Среднее число заявок в системе
    L_system = L_queue + L_busy
    
    # Среднее время ожидания в очереди
    W_queue = L_queue / (lambda_ * (1 - p_refuse)) if lambda_ * (1 - p_refuse) != 0 else 0
    
    # Среднее время пребывания в системе
    W_system = L_system / (lambda_ * (1 - p_refuse)) if lambda_ * (1 - p_refuse) != 0 else 0
    
    return {
        "p0": p0,  # Вероятность простоя
        "p_refuse": p_refuse,  # Вероятность отказа
        "L_queue": L_queue,  # Среднее число заявок в очереди
        "L_system": L_system,  # Среднее число заявок в системе
        "W_queue": W_queue,  # Среднее время ожидания в очереди
        "W_system": W_system  # Среднее время пребывания в системе
    }

def main():
    # Входные параметры
    params = [
        {"S": 2, "lambda_": 0.7, "mu": 1.2, "n": 10},
        {"S": 2, "lambda_": 0.7, "mu": 1.2, "n": 20},
        {"S": 2, "lambda_": 0.3, "mu": 0.5, "n": 10},
        {"S": 2, "lambda_": 0.3, "mu": 0.5, "n": 20},
        {"S": 4, "lambda_": 0.7, "mu": 1.2, "n": 10},
        {"S": 4, "lambda_": 0.7, "mu": 1.2, "n": 20},
        {"S": 4, "lambda_": 0.3, "mu": 0.5, "n": 10},
        {"S": 4, "lambda_": 0.3, "mu": 0.5, "n": 20},
    ]
    
    # Вывод результатов для всех комбинаций
    for param in params:
        result = calculate_queue_metrics(param["S"], param["lambda_"], param["mu"], param["n"])
        print(f"\nПараметры: S={param['S']}, λ={param['lambda_']}, μ={param['mu']}, n={param['n']}")
        print(f"Вероятность простоя (p0): {result['p0']:.6f}")
        print(f"Вероятность отказа: {result['p_refuse']:.6f}")
        print(f"Среднее число заявок в очереди: {result['L_queue']:.6f}")
        print(f"Среднее число заявок в системе: {result['L_system']:.6f}")
        print(f"Среднее время ожидания в очереди: {result['W_queue']:.6f}")
        print(f"Среднее время пребывания в системе: {result['W_system']:.6f}")

if __name__ == "__main__":
    main()


Параметры: S=2, λ=0.7, μ=1.2, n=10
Вероятность простоя (p0): 0.548387
Вероятность отказа: 0.000000
Среднее число заявок в очереди: 0.054236
Среднее число заявок в системе: 0.637569
Среднее время ожидания в очереди: 0.077480
Среднее время пребывания в системе: 0.910813

Параметры: S=2, λ=0.7, μ=1.2, n=20
Вероятность простоя (p0): 0.548387
Вероятность отказа: 0.000000
Среднее число заявок в очереди: 0.054238
Среднее число заявок в системе: 0.637571
Среднее время ожидания в очереди: 0.077483
Среднее время пребывания в системе: 0.910816

Параметры: S=2, λ=0.3, μ=0.5, n=10
Вероятность простоя (p0): 0.538462
Вероятность отказа: 0.000001
Среднее число заявок в очереди: 0.059338
Среднее число заявок в системе: 0.659338
Среднее время ожидания в очереди: 0.197793
Среднее время пребывания в системе: 2.197793

Параметры: S=2, λ=0.3, μ=0.5, n=20
Вероятность простоя (p0): 0.538462
Вероятность отказа: 0.000000
Среднее число заявок в очереди: 0.059341
Среднее число заявок в системе: 0.659341
Среднее 

In [17]:
import math
import random
import numpy as np

def calculate_queue_metrics_analytic(S, lambda_, mu, n):
    """
    Аналитический расчет характеристик СМО M/M/S/n.
    """
    rho = lambda_ / (S * mu)
    p0 = 0
    for k in range(S):
        p0 += (math.pow(S * rho, k) / math.factorial(k))
    for k in range(S, S + n + 1):
        p0 += (math.pow(S * rho, k) / (math.factorial(S) * math.pow(S, k - S)))
    p0 = 1 / p0 if p0 != 0 else 0
    
    p_refuse = (math.pow(S * rho, S + n) / (math.factorial(S) * math.pow(S, n))) * p0
    
    L_queue = 0
    for k in range(1, n + 1):
        L_queue += k * (math.pow(S * rho, S + k) / (math.factorial(S) * math.pow(S, k))) * p0
    
    L_busy = lambda_ * (1 - p_refuse) / mu
    L_system = L_queue + L_busy
    W_queue = L_queue / (lambda_ * (1 - p_refuse)) if lambda_ * (1 - p_refuse) != 0 else 0
    W_system = L_system / (lambda_ * (1 - p_refuse)) if lambda_ * (1 - p_refuse) != 0 else 0
    
    return {
        "p0": p0,
        "p_refuse": p_refuse,
        "L_queue": L_queue,
        "L_system": L_system,
        "W_queue": W_queue,
        "W_system": W_system
    }

def simulate_queue(S, lambda_, mu, n, sim_time=1000, num_trials=100):
    """
    Симуляция СМО M/M/S/n методом Монте-Карло.
    """
    total_refusals = 0
    total_queue_length = 0
    total_system_length = 0
    total_waiting_time = 0
    total_arrivals = 0
    
    for _ in range(num_trials):
        current_time = 0
        queue = []
        servers = [0] * S  # Время, когда каждый сервер освободится
        refusals = 0
        queue_lengths = []
        waiting_times = []
        arrivals = 0
        
        # Генерация первого события
        next_arrival = random.expovariate(lambda_)
        events = [(next_arrival, 'arrival')]
        
        while current_time < sim_time:
            # Сортируем события по времени
            events.sort()
            if not events:
                break
            event_time, event_type = events.pop(0)
            current_time = event_time
            
            if event_type == 'arrival':
                arrivals += 1
                # Проверяем, есть ли свободный сервер
                free_server = -1
                for i in range(S):
                    if servers[i] <= current_time:
                        free_server = i
                        break
                
                if free_server >= 0:
                    # Назначаем заявку на свободный сервер
                    service_time = random.expovariate(mu)
                    servers[free_server] = current_time + service_time
                    events.append((current_time + service_time, 'departure'))
                else:
                    # Проверяем очередь
                    if len(queue) < n:
                        queue.append(current_time)
                    else:
                        refusals += 1
                
                # Генерируем следующее поступление
                next_arrival = current_time + random.expovariate(lambda_)
                events.append((next_arrival, 'arrival'))
            
            elif event_type == 'departure':
                # Если есть заявки в очереди, обслуживаем следующую
                if queue:
                    arrival_time = queue.pop(0)
                    waiting_times.append(current_time - arrival_time)
                    service_time = random.expovariate(mu)
                    events.append((current_time + service_time, 'departure'))
                
                # Обновляем статистику длины очереди
                queue_lengths.append(len(queue))
        
        # Собираем статистику за испытание
        total_refusals += refusals
        total_queue_length += sum(queue_lengths) / len(queue_lengths) if queue_lengths else 0
        total_system_length += (sum(queue_lengths) + sum(1 for s in servers if s > current_time)) / len(queue_lengths) if queue_lengths else 0
        total_waiting_time += sum(waiting_times) / len(waiting_times) if waiting_times else 0
        total_arrivals += arrivals
    
    # Средние значения по всем испытаниям
    p_refuse = total_refusals / total_arrivals if total_arrivals > 0 else 0
    L_queue = total_queue_length / num_trials
    L_system = total_system_length / num_trials
    W_queue = total_waiting_time / num_trials
    
    return {
        "p_refuse": p_refuse,
        "L_queue": L_queue,
        "L_system": L_system,
        "W_queue": W_queue
    }

def main():
    # Входные параметры
    params = [
        {"S": 2, "lambda_": 0.7, "mu": 1.2, "n": 10},
        {"S": 2, "lambda_": 0.7, "mu": 1.2, "n": 20},
        {"S": 2, "lambda_": 0.3, "mu": 0.5, "n": 10},
        {"S": 2, "lambda_": 0.3, "mu": 0.5, "n": 20},
        {"S": 4, "lambda_": 0.7, "mu": 1.2, "n": 10},
        {"S": 4, "lambda_": 0.7, "mu": 1.2, "n": 20},
        {"S": 4, "lambda_": 0.3, "mu": 0.5, "n": 10},
        {"S": 4, "lambda_": 0.3, "mu": 0.5, "n": 20},
    ]
    
    # Вывод результатов
    for param in params:
        print(f"\nПараметры: S={param['S']}, λ={param['lambda_']}, μ={param['mu']}, n={param['n']}")
        
        # Аналитические результаты
        analytic_result = calculate_queue_metrics_analytic(param["S"], param["lambda_"], param["mu"], param["n"])
        print("\nАналитические результаты:")
        print(f"Вероятность простоя (p0): {analytic_result['p0']:.20f}")
        print(f"Вероятность отказа: {analytic_result['p_refuse']:.20f}")
        print(f"Среднее число заявок в очереди: {analytic_result['L_queue']:.6f}")
        print(f"Среднее число заявок в системе: {analytic_result['L_system']:.6f}")
        print(f"Среднее время ожидания в очереди: {analytic_result['W_queue']:.6f}")
        print(f"Среднее время пребывания в системе: {analytic_result['W_system']:.6f}")
        
        # Симуляционные результаты
        sim_result = simulate_queue(param["S"], param["lambda_"], param["mu"], param["n"], sim_time=1000, num_trials=100)
        print("\nСимуляционные результаты (100 испытаний):")
        print(f"Вероятность отказа: {sim_result['p_refuse']:.20f}")
        print(f"Среднее число заявок в очереди: {sim_result['L_queue']:.6f}")
        print(f"Среднее число заявок в системе: {sim_result['L_system']:.6f}")
        print(f"Среднее время ожидания в очереди: {sim_result['W_queue']:.6f}")

if __name__ == "__main__":
    main()


Параметры: S=2, λ=0.7, μ=1.2, n=10

Аналитические результаты:
Вероятность простоя (p0): 0.54838719063736485193
Вероятность отказа: 0.00000041567975859218
Среднее число заявок в очереди: 0.054236
Среднее число заявок в системе: 0.637569
Среднее время ожидания в очереди: 0.077480
Среднее время пребывания в системе: 0.910813

Симуляционные результаты (100 испытаний):
Вероятность отказа: 0.00000000000000000000
Среднее число заявок в очереди: 0.027061
Среднее число заявок в системе: 0.028496
Среднее время ожидания в очереди: 0.512007

Параметры: S=2, λ=0.7, μ=1.2, n=20

Аналитические результаты:
Вероятность простоя (p0): 0.54838709677461172642
Вероятность отказа: 0.00000000000185193946
Среднее число заявок в очереди: 0.054238
Среднее число заявок в системе: 0.637571
Среднее время ожидания в очереди: 0.077483
Среднее время пребывания в системе: 0.910816

Симуляционные результаты (100 испытаний):
Вероятность отказа: 0.00000000000000000000
Среднее число заявок в очереди: 0.026574
Среднее числ